In [9]:
import pandas as pd
import numpy as np
from ngcc_ml import data_tools
from itertools import product

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

# ML
import tensorflow as tf

import keras
from keras import backend as K
from keras import Model
from keras import regularizers
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout

In [2]:
def process_data(df_in, oneHotCols=[], scaledCols=[]):
    df = df_in.copy()
    scaler = StandardScaler()
    oneHotEncoder = OneHotEncoder(categories = "auto", sparse = False)
    if scaledCols != []:
        df[scaledCols] = scaler.fit_transform(df[scaledCols])
    #if oneHotCols != []:
    #    df[encoded_columns] = oneHotEncoder.fit_transform(alldata[oneHotCols])
    return df


def group_split_evaluate(model, df_xy, feature_cols, target_col="BindingEnergy", n_splits=10, group_col="catalyst_name"):
    all_scores, models = [], []
    split_groups = GroupShuffleSplit(test_size=1.0/n_splits, n_splits=n_splits, random_state = 7).split(df_xy, groups=df_xy[group_col])
    model.save_weights("initial_weights.h5")
    for train_inds, test_inds in split_groups:
        model.load_weights("initial_weights.h5")
        train = df_xy.iloc[train_inds]
        test = df_xy.iloc[test_inds]
        X_train_group, y_train_group, X_test_group, y_test_group = data_tools.scale_data(train, test, feature_cols, target_cols)
        model.fit(X_train_group, y_train_group, epochs=150, batch_size=16)
        mae = model.evaluate(X_test_group, y_test_group)
        all_scores.append(mae)
        models.append(model)
        print('MAE on test set: {:.2f}'.format(mae))
    print("mean:", np.mean(all_scores))
    return all_scores


def create_ffnn_regression(inp_dim, all_lsize, l2_reg=0.0001, drop_val = 0.1):
    model = Sequential()
    model.add(Dense(all_lsize[0], input_dim=inp_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg)))
    model.add(Dropout(drop_val))
    for lsize in all_lsize[1:]:
        model.add(Dense(lsize, activation='relu', kernel_initializer="normal", kernel_regularizer=regularizers.l2(l2_reg)))
        model.add(Dropout(drop_val))
    model.add(Dense(1, kernel_initializer="normal"))
    return model


def sample_list(inlist):
    return inlist[np.random.randint(len(inlist))]


def sample_nested_list(inlist):
    sampled = []
    for sublist in inlist:
        sampled.append(sample_list(sublist))
    return sampled


def hyperparam_sample(df_xy, feature_cols, lsize_opts, l2_reg_opts, drop_val_opts, n_splits=5, hyperparam_samples=6):
    
    lsize_list, l2_reg_list, drop_val_list, group_scores_list = [], [], [], []
    for i in range(hyperparam_samples):
        lsize_sample = sample_nested_list(lsize_opts)
        l2_reg_sample = sample_list(l2_reg_opts)
        drop_val_sample = sample_list(drop_val_opts)   
        model = create_ffnn_regression(len(feature_cols), lsize_sample, l2_reg_sample, drop_val_sample)
        model.compile(loss="mean_absolute_error", optimizer="adam")
        group_scores = group_split_evaluate(model, df_xy, feature_cols, n_splits=n_splits)

        lsize_list.append(lsize_sample)
        l2_reg_list.append(l2_reg_sample)
        drop_val_list.append(drop_val_sample)
        group_scores_list.append(group_scores)
    
    df_results = pd.DataFrame({"lsize":lsize_list, "l2_reg":l2_reg_list, "drop_val":drop_val_list, "scores":group_scores_list})
    return df_results


def hyperparam_grid(df_xy, feature_cols, lsize_opts, l2_reg, drop_val, target_col="BindingEnergy", n_splits=5):
    
    lsize_list, l2_reg_list, drop_val_list, group_scores_list = [], [], [], []
    for layer in product(*lsize_opts):
        lsize = sample_nested_list(lsize_opts)   
        model = create_ffnn_regression(len(feature_cols), lsize, l2_reg, drop_val)
        model.compile(loss="mean_absolute_error", optimizer="adam")
        group_scores = group_split_evaluate(model, df_xy, feature_cols, target_col=target_col, n_splits=n_splits)

        lsize_list.append(lsize)
        l2_reg_list.append(l2_reg)
        drop_val_list.append(drop_val)
        group_scores_list.append(group_scores)
    
    df_results = pd.DataFrame({"lsize":lsize_list, "l2_reg":l2_reg_list, "drop_val":drop_val_list, "scores":group_scores_list})
    return df_results

In [3]:
df = pd.read_csv("/home/nricke/work/ngcc_ml/DidItBindv5.csv", index_col=0)
df.rename(columns={"Catalyst Name":"catalyst_name"}, inplace=True)
df["Doesitbind"] = df["Doesitbind"].astype("int")
#df = df[df["Doesitbind"] == 1]

In [4]:
feature_cols = {"SpinDensity", "ChElPGNeutralCharge", "ChargeDifference", "IonizationEnergy", "OrthoOrPara", "Meta", "FartherThanPara", "DistanceToN", "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "NeighborSpinDensity", 'NeighborChElPGCharge', 'NeighborChargeDifference', "AromaticExtent", "RingEdge", "NumNitrogens", "NumHeteroatoms", "charge", "atom_plane_deviation", "ring_plane_deviation", "ring_nitrogens"}
not_scaled_cols = {"OrthoOrPara", "Meta", "FartherThanPara", "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "RingEdge", "NumNitrogens", "NumHeteroatoms", "ring_nitrogens", "charge"}

In [6]:
target_cols = ["Doesitbind", "BindingEnergy"]

df_active = df[df["Doesitbind"] == 1]
df_spec = df.copy()
df_spec.loc[df_spec['Doesitbind'] == 0, 'BindingEnergy'] = 0.0

In [7]:
all_lsize = [1024, 512, 256, 128, 16]
drop_val = 0.1
l2_reg = 0.00001
inp_dim = len(feature_cols)

In [8]:
target_cols = ["BindingEnergy"]

visible = Input(shape=inp_dim)
hidden = Dense(all_lsize[0], activation="relu", kernel_initializer="normal", kernel_regularizer=keras.regularizers.l2(l2_reg))(visible)
hidden = Dropout(drop_val)(hidden)
for lsize in all_lsize[1:]:
    hidden = Dense(lsize, activation='relu', kernel_initializer="normal", kernel_regularizer=regularizers.l2(l2_reg))(hidden)
    hidden = Dropout(drop_val)(hidden)
be_reg = Dense(1, name="BindingEnergy", kernel_initializer="normal")(hidden)
#dib_class = Dense(1, name="Doesitbind", activation="sigmoid")(hidden)
model = Model(inputs=visible, outputs=[be_reg])
model.compile(loss={
                    #"Doesitbind": "binary_crossentropy", 
                    "BindingEnergy":"mean_absolute_error"}, 
              optimizer="adam",
              metrics={"BindingEnergy":"mean_absolute_error"},
             )

evv = []
split_groups = GroupKFold(n_splits=10).split(df_active[feature_cols], df_active[target_cols], df_active["catalyst_name"])
model.save_weights("initial_weights.h5")
for train_inds, test_inds in split_groups:
    model.load_weights("initial_weights.h5")
    train = df_active.iloc[train_inds]
    test = df_active.iloc[test_inds]
    X_train_group, y_train_group, X_test_group, y_test_group = data_tools.scale_data(train, test, feature_cols, target_cols)
    model.fit(X_train_group, {"BindingEnergy": y_train_group["BindingEnergy"]}, 
              epochs=100, batch_size=16)
    evv.append(model.evaluate(X_test_group, {"BindingEnergy": y_test_group["BindingEnergy"]}))

Epoch 1/100
42/42 [==============================] - 0s 4ms/step - loss: 0.2387 - mean_absolute_error: 0.2213
Epoch 2/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1320 - mean_absolute_error: 0.1157
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1296 - mean_absolute_error: 0.1145
Epoch 4/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1172 - mean_absolute_error: 0.1031
Epoch 5/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1204 - mean_absolute_error: 0.1070
Epoch 6/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1080 - mean_absolute_error: 0.0954
Epoch 7/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1111 - mean_absolute_error: 0.0990
Epoch 8/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1097 - mean_absolute_error: 0.0981
Epoch 9/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1012 - mean_absolute_error: 0.0900
Epoch 10/1

42/42 [==============================] - 0s 4ms/step - loss: 0.0561 - mean_absolute_error: 0.0507
Epoch 75/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0556 - mean_absolute_error: 0.0502
Epoch 76/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0524 - mean_absolute_error: 0.0470
Epoch 77/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0550 - mean_absolute_error: 0.0496
Epoch 78/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0572 - mean_absolute_error: 0.0519
Epoch 79/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0590 - mean_absolute_error: 0.0537
Epoch 80/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0540 - mean_absolute_error: 0.0487
Epoch 81/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0511 - mean_absolute_error: 0.0458
Epoch 82/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0514 - mean_absolute_error: 0.0461
Epoch 83/100
4

42/42 [==============================] - 0s 4ms/step - loss: 0.0642 - mean_absolute_error: 0.0586
Epoch 48/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0669 - mean_absolute_error: 0.0613
Epoch 49/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0627 - mean_absolute_error: 0.0572
Epoch 50/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0611 - mean_absolute_error: 0.0556
Epoch 51/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0659 - mean_absolute_error: 0.0604
Epoch 52/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0617 - mean_absolute_error: 0.0563
Epoch 53/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0591 - mean_absolute_error: 0.0538
Epoch 54/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0642 - mean_absolute_error: 0.0589
Epoch 55/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0621 - mean_absolute_error: 0.0568
Epoch 56/100
4

42/42 [==============================] - 0s 4ms/step - loss: 0.0842 - mean_absolute_error: 0.0775
Epoch 21/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0806 - mean_absolute_error: 0.0740
Epoch 22/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0776 - mean_absolute_error: 0.0712
Epoch 23/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0791 - mean_absolute_error: 0.0727
Epoch 24/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0789 - mean_absolute_error: 0.0726
Epoch 25/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0793 - mean_absolute_error: 0.0731
Epoch 26/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0738 - mean_absolute_error: 0.0678
Epoch 27/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0776 - mean_absolute_error: 0.0716
Epoch 28/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0719 - mean_absolute_error: 0.0660
Epoch 29/100
4

42/42 [==============================] - 0s 4ms/step - loss: 0.0520 - mean_absolute_error: 0.0471
Epoch 94/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0495 - mean_absolute_error: 0.0446
Epoch 95/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0528 - mean_absolute_error: 0.0478
Epoch 96/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0520 - mean_absolute_error: 0.0471
Epoch 97/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0518 - mean_absolute_error: 0.0469
Epoch 98/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0511 - mean_absolute_error: 0.0461
Epoch 99/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0493 - mean_absolute_error: 0.0443
Epoch 100/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0691 - mean_absolute_error: 0.0642
Epoch 1/100
42/42 [==============================] - 0s 4ms/step - loss: 0.2077 - mean_absolute_error: 0.1901
Epoch 2/100
42/4

...

42/42 [==============================] - 0s 4ms/step - loss: 0.0585 - mean_absolute_error: 0.0530
Epoch 59/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0591 - mean_absolute_error: 0.0536
Epoch 60/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0574 - mean_absolute_error: 0.0519
Epoch 61/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0600 - mean_absolute_error: 0.0545
Epoch 62/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0588 - mean_absolute_error: 0.0534
Epoch 63/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0571 - mean_absolute_error: 0.0517
Epoch 64/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0596 - mean_absolute_error: 0.0542
Epoch 65/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0600 - mean_absolute_error: 0.0547
Epoch 66/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0611 - mean_absolute_error: 0.0558
Epoch 67/100
4

42/42 [==============================] - 0s 4ms/step - loss: 0.0719 - mean_absolute_error: 0.0655
Epoch 32/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0711 - mean_absolute_error: 0.0648
Epoch 33/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0714 - mean_absolute_error: 0.0651
Epoch 34/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0726 - mean_absolute_error: 0.0664
Epoch 35/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0669 - mean_absolute_error: 0.0608
Epoch 36/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0705 - mean_absolute_error: 0.0644
Epoch 37/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0672 - mean_absolute_error: 0.0612
Epoch 38/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0672 - mean_absolute_error: 0.0612
Epoch 39/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0647 - mean_absolute_error: 0.0588
Epoch 40/100
4

42/42 [==============================] - 0s 4ms/step - loss: 0.1226 - mean_absolute_error: 0.1091
Epoch 5/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1242 - mean_absolute_error: 0.1116
Epoch 6/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1082 - mean_absolute_error: 0.0963
Epoch 7/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1160 - mean_absolute_error: 0.1047
Epoch 8/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1076 - mean_absolute_error: 0.0968
Epoch 9/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1033 - mean_absolute_error: 0.0929
Epoch 10/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1041 - mean_absolute_error: 0.0941
Epoch 11/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0951 - mean_absolute_error: 0.0855
Epoch 12/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0906 - mean_absolute_error: 0.0813
Epoch 13/100
42/42 

Epoch 78/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0547 - mean_absolute_error: 0.0495
Epoch 79/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0564 - mean_absolute_error: 0.0512
Epoch 80/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0544 - mean_absolute_error: 0.0492
Epoch 81/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0561 - mean_absolute_error: 0.0509
Epoch 82/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0571 - mean_absolute_error: 0.0520
Epoch 83/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0536 - mean_absolute_error: 0.0485
Epoch 84/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0554 - mean_absolute_error: 0.0503
Epoch 85/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0548 - mean_absolute_error: 0.0497
Epoch 86/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0531 - mean_absolute_error: 0.0481
E

42/42 [==============================] - 0s 4ms/step - loss: 0.0594 - mean_absolute_error: 0.0538
Epoch 52/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0584 - mean_absolute_error: 0.0528
Epoch 53/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0579 - mean_absolute_error: 0.0524
Epoch 54/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0573 - mean_absolute_error: 0.0518
Epoch 55/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0549 - mean_absolute_error: 0.0494
Epoch 56/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0579 - mean_absolute_error: 0.0524
Epoch 57/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0609 - mean_absolute_error: 0.0554
Epoch 58/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0585 - mean_absolute_error: 0.0531
Epoch 59/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0576 - mean_absolute_error: 0.0522
Epoch 60/100
4

In [10]:
evv

[[0.08523955196142197, 0.0800771564245224],
 [0.11088090389966965, 0.10631784051656723],
 [0.06910989433526993, 0.06415044516324997],
 [0.08883629739284515, 0.08380763232707977],
 [0.07366611063480377, 0.0686270147562027],
 [0.08392292261123657, 0.07904312759637833],
 [0.06094859167933464, 0.05598107725381851],
 [0.09322825074195862, 0.08819302916526794],
 [0.07739178091287613, 0.07243216782808304],
 [0.0897667184472084, 0.08484796434640884]]

In [11]:
evv = np.array(evv)
np.mean(evv[:,1])

0.07834774553775788

In [12]:
y_true = [1]
y_pred = [1]
mse = tf.keras.losses.MeanSquaredError(
    reduction=tf.keras.losses.Reduction.SUM)
mse(y_true, y_pred).numpy()

0

In [13]:
mask_value = 0

def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
    #return K.binary_crossentropy(y_true * mask, y_pred * mask)
    #return K.mean(K.square((y_pred-y_true) * mask))
    masked_square_error = K.square((y_pred-y_true) * mask)

    
# https://github.com/keras-team/keras/issues/7065
def masked_mse(mask_value):
    def f(y_true, y_pred):
        mask_true = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
        return masked_mse
    #f.__name__ = 'Masked MSE (mask_value={})'.format(mask_value)
    f__name__ = "masked_mse_0"
    return f

masked_mse0 = masked_mse(0)


In [15]:
masked_mse0(0, -0.5)

InvalidArgumentError: Invalid reduction dimension (-1 for input with 0 dimension(s) [Op:Sum]

In [14]:
target_cols = ["Doesitbind", "BindingEnergy"]

visible = Input(shape=inp_dim)
hidden = Dense(all_lsize[0], activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(visible)
hidden = Dropout(drop_val)(hidden)
for lsize in all_lsize[1:]:
    hidden = Dense(lsize, activation='relu', kernel_initializer="normal", kernel_regularizer=regularizers.l2(l2_reg))(hidden)
    hidden = Dropout(drop_val)(hidden)
be_reg = Dense(1, name="BindingEnergy", kernel_initializer="normal")(hidden)
dib_class = Dense(1, name="Doesitbind", activation="sigmoid")(hidden)
model = Model(inputs=visible, outputs=[dib_class, be_reg])
model.compile(loss={
                    "Doesitbind": "binary_crossentropy",
                    "BindingEnergy": masked_mse0,
                    #"BindingEnergy":"mean_absolute_error"
                    }, 
              optimizer="adam",
              metrics={"Doesitbind":"accuracy", "BindingEnergy":"mean_absolute_error"},
             )
    
evv = []
split_groups = GroupKFold(n_splits=3).split(df_spec[feature_cols], df_spec[target_cols], df_spec["catalyst_name"])
model.save_weights("initial_weights.h5")
for train_inds, test_inds in split_groups:
    model.load_weights("initial_weights.h5")
    train = df_spec.iloc[train_inds]
    test = df_spec.iloc[test_inds]
    X_train_group, y_train_group, X_test_group, y_test_group = data_tools.scale_data(train, test, feature_cols, target_cols)
    model.fit(X_train_group, {"Doesitbind": y_train_group["Doesitbind"], "BindingEnergy": y_train_group["BindingEnergy"]}, 
              epochs=100, batch_size=16)
    evv.append(model.evaluate(X_test_group, {"Doesitbind": y_test_group["Doesitbind"], "BindingEnergy": y_test_group["BindingEnergy"]}))

Epoch 1/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.6682 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8170 - BindingEnergy_mean_absolute_error: nan
Epoch 2/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.6252 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 3/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.5906 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 4/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.5630 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 5/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.5414 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_e

173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 42/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 43/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 44/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 45/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: na

173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 82/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 83/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 84/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: nan
Epoch 85/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4743 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8181 - BindingEnergy_mean_absolute_error: na

173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4598 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 21/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 22/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4598 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 23/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 24/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: na

173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 61/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 62/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 63/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: nan
Epoch 64/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4597 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8276 - BindingEnergy_mean_absolute_error: na

44/44 [==============================] - 0s 3ms/step - loss: nan - Doesitbind_loss: 0.4882 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8094 - BindingEnergy_mean_absolute_error: nan
Epoch 1/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.6388 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8171 - BindingEnergy_mean_absolute_error: nan
Epoch 2/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.5690 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 3/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.5337 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 4/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.5127 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoc

173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 40/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 41/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 42/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 43/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: na

173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 80/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 81/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 82/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: nan
Epoch 83/100
173/173 [==============================] - 1s 5ms/step - loss: nan - Doesitbind_loss: 0.4731 - BindingEnergy_loss: nan - Doesitbind_accuracy: 0.8189 - BindingEnergy_mean_absolute_error: na

In [ ]:
np.array(evv)

In [ ]:
model.evaluate(X_test_group, {"Doesitbind": y_test_group["Doesitbind"], "BindingEnergy": y_test_group["BindingEnergy"]})

In [ ]:
model.predict(X_test_group)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test_group, y_pred)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.annotate("R$^2$ = 0.83", xy=(-1,-0.1))
ax.annotate("10-fold CV MAE = 0.080 eV", xy=(-1,-0.145))
plt.plot([-1.1,0],[-1.1,0.], color="black")
plt.ylim([-1.1,0])
plt.xlim([-1.1,0])
plt.ylabel(r"O$_2$ Binding Energy (eV) Predicted")
plt.xlabel(r"O$_2$ Binding Energy (eV) DFT")
plt.scatter(y_test_group, y_pred, s=150)
#plt.show()
plt.savefig("Regression_R2_MAE.png")

In [ ]:
model.fit(X_train_group, y_train_group, epochs=200, batch_size=16, class_weight={0:4, 1:1})

In [ ]:
_, accuracy = model.evaluate(X_test_group, y_test_group)
print('Accuracy: %.2f' % (accuracy*100))